# Experiment 02 

### General description 

Mix Jigsaw dataset and CTEC dataset for training. 

<ul>
    <li>Encoding: TF-IDF</li>
    <li>Models: logistic regression vs. multinomial bayesian</li>
    <li>Training set: 18386 jigsaw negative + 1614 jigsaw positive + 10000 ctec positive</li>
    <li>Test set: 1901 ctec negative + 11600 ctec positive</li>
    <li>Metric: ROC AUC score</li>
</ul>

## Load data

In [10]:
import pandas as pd
import numpy as np 
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

In [2]:
# Load jigsaw and ctec datasets 
jigsaw_df = pd.read_csv('train_preproc_shrk.csv')
ctec_df = pd.read_csv('ctec_training_data_preproc.csv')

# target >= 0.5 --> toxic --> label = 1
# target < 0.5 --> non-toxic --> label = 0
jigsaw_df.loc[jigsaw_df['target'] >= 0.5, 'label'] = 1
jigsaw_df.loc[jigsaw_df['target'] < 0.5, 'label'] = 0

# Split by label 
jigsaw_neg = jigsaw_df[jigsaw_df['label'] == 0]
jigsaw_pos = jigsaw_df[jigsaw_df['label'] == 1]
ctec_neg = ctec_df[ctec_df['label'] == 0]
ctec_pos = ctec_df[ctec_df['label'] == 1]

# Show number of positive and negative examples in each dataset 
print(f'jigsaw # negative examples = {jigsaw_neg.shape[0]}')
print(f'jigsaw # positive examples = {jigsaw_pos.shape[0]}')
print(f'ctec # negative examples = {ctec_neg.shape[0]}')
print(f'ctec # positive examples = {ctec_pos.shape[0]}')

jigsaw # negative examples = 18386
jigsaw # positive examples = 1614
ctec # negative examples = 1901
ctec # positive examples = 21600


We determine the scheme of data combination and train-test-split based on the proportion of positive and negative examples. 

For now, we take the truncated jigsaw dataset together with 10,000 positive examples of the ctec dataset as the training set. The remaining ctec dataset (11600 positive and 1901 negative) is the test set. In this way, we are able to reach a better balance of the training set. 

In [3]:
# Create training set and test set based on the scheme described above

# Randomly sampling indices 
ind = np.random.choice(range(ctec_pos.shape[0]), size = 10000, replace = False)
notind = np.setdiff1d(range(ctec_pos.shape[0]), ind)

# X_train in text format, and y_train
# All jigsaw examples plus sampled ctec positive examples
X_train_text = jigsaw_df['comment_text'].append(
    ctec_pos['comment_text'].iloc[ind], 
    ignore_index = True
)

y_train = jigsaw_df['label'].append(
    ctec_pos['label'].iloc[ind], 
    ignore_index = True
)

# X_test in text format, and y_test 
X_test_text = ctec_neg['comment_text'].append(
    ctec_pos['comment_text'].iloc[notind], 
    ignore_index = True
)

y_test = ctec_neg['label'].append(
    ctec_pos['label'].iloc[notind], 
    ignore_index = True
)


### Vectorization and normalization

In [4]:
# Vectorization with tfidf
encoder = TfidfVectorizer(strip_accents = 'unicode', stop_words = 'english')
X_train_unscaled = encoder.fit_transform(X_train_text)
X_test_unscaled = encoder.transform(X_test_text)

# Normalization without 0-mean
scaler = StandardScaler(with_mean = False)
X_train = scaler.fit_transform(X_train_unscaled)
X_test = scaler.transform(X_test_unscaled)

## Training and testing 

### Multinomial naive Bayes

In [11]:
# Model and hyperparameterization
clf = GridSearchCV(
    MultinomialNB(), 
    param_grid = {'alpha': [0.1, 1, 10]}, 
    scoring = 'roc_auc'
)

# Train 
%time clf.fit(X_train, y_train)
 
# Predict label
y_train_pred_class = clf.predict(X_train)
y_test_pred_class = clf.predict(X_test)
# Predict probability of being toxic
y_train_pred_prob = clf.predict_proba(X_train)[:,1]
y_test_pred_prob = clf.predict_proba(X_test)[:,1]

print('\nMultinomial naive Bayes')
print(f'best parameter = {clf.best_params_}')

# Store results
results = [
    [
        'accuracy', 
        metrics.accuracy_score(y_train, y_train_pred_class), 
        metrics.accuracy_score(y_test, y_test_pred_class)
    ], 
    [
        'confusion matrix', 
        str(metrics.confusion_matrix(y_train, y_train_pred_class).tolist()), 
        str(metrics.confusion_matrix(y_test, y_test_pred_class).tolist())
    ], 
    [
        'F1 score', 
        metrics.f1_score(y_train, y_train_pred_class), 
        metrics.f1_score(y_test, y_test_pred_class)
    ], 
    [
        'ROC AUC score', 
        metrics.roc_auc_score(y_train, y_train_pred_class), 
        metrics.roc_auc_score(y_test, y_test_pred_class)
    ]
]

colNames = ['metric', 'train set', 'test set']

# Show result 
pd.DataFrame(results, columns = colNames)

CPU times: user 397 ms, sys: 979 µs, total: 398 ms
Wall time: 418 ms

Multinomial naive Bayes
best parameter = {'alpha': 10}


,metric,train set,test set
0,accuracy,0.932767,0.652174
1,confusion matrix,"[[18292, 94], [1923, 9691]]","[[811, 1090], [3606, 7994]]"
2,F1 score,0.905743,0.772965
3,ROC AUC score,0.914656,0.557878


## Logistic regression

In [12]:
# Model and hyperparameterization
clf = GridSearchCV(
    LogisticRegression(max_iter = 2000), 
    param_grid = {'C': [0.1, 1, 10]}, 
    scoring = 'roc_auc'
)

# Train 
%time clf.fit(X_train, y_train)
 
# Predict label
y_train_pred_class = clf.predict(X_train)
y_test_pred_class = clf.predict(X_test)
# Predict probability of being toxic
y_train_pred_prob = clf.predict_proba(X_train)[:,1]
y_test_pred_prob = clf.predict_proba(X_test)[:,1]

print('\nLogistic regression')
print(f'best parameter = {clf.best_params_}')

# Store results
results = [
    [
        'accuracy', 
        metrics.accuracy_score(y_train, y_train_pred_class), 
        metrics.accuracy_score(y_test, y_test_pred_class)
    ], 
    [
        'confusion matrix', 
        str(metrics.confusion_matrix(y_train, y_train_pred_class).tolist()), 
        str(metrics.confusion_matrix(y_test, y_test_pred_class).tolist())
    ], 
    [
        'F1 score', 
        metrics.f1_score(y_train, y_train_pred_class), 
        metrics.f1_score(y_test, y_test_pred_class)
    ], 
    [
        'ROC AUC score', 
        metrics.roc_auc_score(y_train, y_train_pred_class), 
        metrics.roc_auc_score(y_test, y_test_pred_class)
    ]
]

colNames = ['metric', 'train set', 'test set']

# Show result 
pd.DataFrame(results, columns = colNames)

CPU times: user 1min 55s, sys: 1min 27s, total: 3min 22s
Wall time: 52.4 s

Logistic regression
best parameter = {'C': 0.1}


,metric,train set,test set
0,accuracy,0.998133,0.739649
1,confusion matrix,"[[18380, 6], [50, 11564]]","[[390, 1511], [2004, 9596]]"
2,F1 score,0.997585,0.845202
3,ROC AUC score,0.997684,0.516198
